In [ ]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [2]:
# cache the csv data
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv')

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

In [3]:

df = df[df["year"] >= 1980]
df = df[df["year"] < 2023]
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']
# df = df[df["country"].isin(continents)]
df

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,ASEAN (Ember),2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
1,ASEAN (Ember),2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
2,ASEAN (Ember),2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
3,ASEAN (Ember),2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
4,ASEAN (Ember),2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21743,Zimbabwe,2018,ZWE,15052191.0,2.678627e+10,NaN,NaN,NaN,NaN,12.623,...,0.212,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21744,Zimbabwe,2019,ZWE,15354606.0,2.514642e+10,NaN,NaN,NaN,NaN,12.374,...,0.353,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21745,Zimbabwe,2020,ZWE,15669663.0,2.317871e+10,NaN,NaN,NaN,NaN,6.382,...,0.298,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21746,Zimbabwe,2021,ZWE,15993525.0,2.514009e+10,NaN,NaN,NaN,NaN,0.000,...,0.236,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN


## (0) Data preprocessing

In [4]:
# Fill NAs with 0s and create GDP per capita column
df = df.fillna(0)
df['gdp_per_capita'] = np.where(df['population']!= 0, df['gdp']/ df['population'], 0)


In [5]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

## (1) Energy consumption over time by continent

In [6]:
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=1980, end=2022, step=1, value=2020)
year_slider

IntSlider(end=2022, name='Year slider', start=1980, value=2020)

In [7]:
# Radio buttons for Energy Consumption
yaxis_energy = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['primary_energy_consumption', 'energy_per_capita',],
    button_type='success'
)

In [8]:
energy_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_energy].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [9]:
energy_pipeline

In [10]:
energy_plot = energy_pipeline.hvplot(x = 'year', by='country', y=yaxis_energy,line_width=2, title="energy consumption by continent")
energy_plot

## (2) Table - energy consumption over time by continent 

In [11]:
energy_table = energy_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
energy_table

## (3) Energy vs GDP scatterplot

In [12]:
energy_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~ (idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['primary_energy_consumption'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [13]:
energy_vs_gdp_scatterplot_pipeline

In [14]:
energy_vs_gdp_scatterplot = energy_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita', 
                                                                y='primary_energy_consumption', 
                                                                by='country', 
                                                                size=20, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
energy_vs_gdp_scatterplot

## (4) Bar chart with energy sources by continent

In [15]:
yaxis_energy_source = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['biofuel_consumption', 'coal_consumption', 'fossil_fuel_consumption', 'gas_consumption', 'hydro_consumption', 'low_carbon_consumption', 'nuclear_consumption', 'solar_consumption', 'wind_consumption'], 
    button_type='success'
)

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

energy_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_energy_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [16]:
energy_source_bar_plot = energy_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='country', 
                                                     y=yaxis_energy_source, 
                                                     title='energy source by continent')
energy_source_bar_plot

## Creating Dashboard

In [17]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='World Energy Dashboard', 
    
    main=[pn.Row(pn.Column(yaxis_energy,energy_table.panel(width=500),energy_plot.panel(width=600), margin=(0,25))), 
          pn.Row(pn.Column(energy_vs_gdp_scatterplot.panel(width=600), margin=(0,25)),pn.Column(yaxis_energy_source, energy_source_bar_plot.panel(width=600)))],
    sidebar=[pn.pane.Markdown("# Energy Consumption"), 
             pn.pane.Markdown("#### "), 
             pn.pane.Markdown("## Settings"),   
             year_slider],
    accent_base_color="#FFC300",
    header_background="#85C1E9",
)
# template.show()
template.servable();